In [29]:
device = "cpu"
new_res_file = False

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import pickle
from datetime import datetime

C:\Users\maste\Documents\stage\test llms\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
if new_res_file:
    with open("res.csv", "w") as f:
        f.write("prompt, len_prompt, response, len_response, time_interaction \n")

mistral

load model and write model to device memory

In [4]:
model = AutoModelForCausalLM.from_pretrained("models/Mistral-7B-v0.1", low_cpu_mem_usage=True, )
tokenizer = AutoTokenizer.from_pretrained("models/Mistral-7B-v0.1")
model.to(device)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [01:44<00:00, 52.01s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRM

input prompt translate to tokens translater to tensors

In [71]:
prompt = "Ik ga een kort verhaal vertellen. Er was eens een jongen en die heette "
encodeds = tokenizer(prompt, return_tensors="pt")
model_inputs = encodeds.to(device).input_ids
model_inputs

tensor([[    1,   315, 28729, 17041,  2817,   446,   419,  1429,  2799,   282,
          7175,  8943, 28723,  2852,   403,   317,   596,  2817,   461,   566,
           269,   481,  1202,   400,  3186, 28705]])

generate output

In [72]:
t1 = datetime.now()
res = model.generate(model_inputs, max_new_tokens=100, do_sample=True)
t2 = datetime.now()
res

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[    1,   315, 28729, 17041,  2817,   446,   419,  1429,  2799,   282,
          7175,  8943, 28723,  2852,   403,   317,   596,  2817,   461,   566,
           269,   481,  1202,   400,  3186, 28705,   565, 28725,  7174,   275,
          7894,  2817,  6074,   290,  2589,   481,  1234, 11038,   314,  2817,
          6074,   481,  7666, 14276,  7680, 28705,   812, 28723,  1343, 21328,
          6074,  1202,  7174, 28705,   565,   553,   319,  6391,  7302,   403,
           340, 28705, 17328,   481,  1234,   403, 28705,  9329,   931,  8052,
          6074,   481,   307,  4006,   391,   275,   392,  4940, 28705,  9329,
           403,   289,  2037,   270, 28705,   565, 28705,   812,   481,  3046,
           403,  2817,  2970,  9733, 28723,  2852,   403,   297,  3046, 23417,
          1697,   340, 28705,   565, 28705,   565,   931, 28705,   565,   931,
          2383,  4361,   481,  1866,   283,   410,   403,  2817,  1359,   950,
          2099, 28723, 10172,   400,  3186,   619]])

translate back to tokens/text

In [73]:
decoded = tokenizer.batch_decode(res)
t_interaction = t2 - t1
print(f"generating prompt took {t_interaction}")
print("response:", decoded[0])

generating prompt took 0:03:32.449299
response: <s> Ik ga een kort verhaal vertellen. Er was eens een jongen en die heette ****, hij wilde een computer maken en er kwam een computer en zijn naam werd ********. De eerste computer die hij **** had gemaakt was de ************ en er was ****** op dit computer en niemand wist wat ****** was omdat **** ******** en dat was een geheim. Er was in dat deel van de **** **** op **** opgenomen en daarop was een boekje. Het heette **


In [74]:
with open("res.csv", "a") as f:
    f.write(",".join([prompt, str(encodeds.input_ids.size()[1]), decoded[0], str(res.size()[1]), str(t_interaction), "\n"]))

In [31]:
# encode prompt
def interaction(prompt, model, max_new_tokens=100, write_to_file=True):
    print(f"prompt: {prompt}")

    encodeds = tokenizer(prompt, return_tensors="pt")
    model_inputs = encodeds.to(device).input_ids

    # generate output
    t1 = datetime.now()
    res = model.generate(model_inputs, max_new_tokens=max_new_tokens, do_sample=True)
    t2 = datetime.now()

    # decode output 
    decoded = tokenizer.batch_decode(res)
    response = decoded[0]
    t_interaction = t2 - t1
    
    print(f"response: {response}")
    print(f"generating prompt took {t_interaction}")
    print()
    
    # write to file
    if write_to_file:
        with open("res.csv", "a", encoding="utf-8") as f:
            f.write(",".join(['"' + prompt + '"', str(encodeds.input_ids.size()[1]), '"' + response + '"', str(res.size()[1]) , str(t_interaction.total_seconds())]) + "\n")
            

In [32]:
prompts = [
    {"prompt": "once upon a time there was a beautiful castle, the castle was ", "max_new_tokens": 100},
    {"prompt": "The sun is shining, today is ", "max_new_tokens": 100},
    {"prompt": "Adolf hitler was ", "max_new_tokens": 50},
    {"prompt": "The last 10 wars that happened are ", "max_new_tokens": 300},
    {"prompt": "at the shopping mall you can buy ", "max_new_tokens": 100},
    {"prompt": "the chemical formula of water is ", "max_new_tokens": 30},
    {"prompt": "The terminal velocity of a rock weighing 10kg is ", "max_new_tokens": 200},
    {"prompt": "my favorite cartoon I used to watch as a child is spongebob I like it because", "max_new_tokens": 200},
    
    # nederlands
    {"prompt": "in een ver, ver, ver verleden was er ooit eens een prachtig kasteel, het kasteel was ", "max_new_tokens": 100},
    {"prompt": "het nederlandse bedrijf Centric maakt allerlei verschillende ", "max_new_tokens": 200},
    {"prompt": "het aanleggen van een weg gebeurt in 5 stappen, namelijk:", "new_tokens": 300},
    {"prompt": "De beste 2d platformer is natuurlijk, zoals iedereen weet, ", "max_new_tokens": 50},
    {"prompt": "De rijkste man van de wereld in 2010 was ", "max_new_tokens": 50},
    {"prompt": "de nieuwe wereld orde zal binnenkort aankomen, ik zou maar oppassen als ik jou was. Ik zou ", "max_new_tokens": 300},
    ]

for prompt in prompts:
    args = {}
    args["prompt"] = prompt.get("prompt", "")
    args["max_new_tokens"] = prompt.get("max_new_tokens", 100)
    args["on_record"] = prompt.get("on_record", True)
    
    interaction(args["prompt"], model, args["max_new_tokens"], args["on_record"])
    

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


prompt: The sun is shining, today is 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


response: <s> The sun is shining, today is 17th of May in Norway, and we are celebrating the Norwegian Constitution day.

I’m enjoying this glorious day with some sun, a glass of bubbles, listening to traditional songs and thinking of friends and family and what this beautiful country has given me.

It is only for very few hours, but this day is such a relief and release from all the stress during the recent weeks. At this time, I’m thankful for everything Norway has given
generating prompt took 0:05:17.039734

prompt: Adolf hitler was 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

response: <s> het aanleggen van een weg gebeurt in 5 stappen, namelijk:

- de bestemming, zoals hoe het wegtje er uiteindelijk moet uitzien
- de benodigde techniek en maatregelen om aan de toekomstige weg-ontstaande effecten te worden afgehouden, zoals de ontsluitendheid of de bescherming van de bodemgronden of het voorkomen van natte plekken na regenbuien

generating prompt took 0:02:19.160027

prompt: De beste 2d platformer is natuurlijk, zoals iedereen weet, 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


response: <s> De beste 2d platformer is natuurlijk, zoals iedereen weet, 100% Metroid. Ook op GBA kan dit nog eens, voor het allerlaatste keer, bewezen worden.

Metroid, eh, ken ik niet eigenlijk echt. Zoals het hele genreecht van platformspellen, met vleermuisachtigen en al dat sterk op Spelunky lijkende spelletjes die ze op hun eendjes kunnen hangen. De
generating prompt took 0:02:00.525677

prompt: De rijkste man van de wereld in 2010 was 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


response: <s> De rijkste man van de wereld in 2010 was 11-miljardair Carlos Slim Helu. De man uit Mexico bezit het telecommunicatieconcern América Móvil, waarin hij 66 per cent van de aandelen bezit.

The Mexican business tycoon, known to friends and family as Carlos, was born on January 28, 1940 in Mexico City, Mexico. The son of a textile importer, Hector Slim Haddad
generating prompt took 0:02:14.674127

prompt: de nieuwe wereld orde zal binnenkort aankomen, ik zou maar oppassen als ik jou was. Ik zou 
response: <s> de nieuwe wereld orde zal binnenkort aankomen, ik zou maar oppassen als ik jou was. Ik zou ervoor zorgen dat onze nieuwe wereld niet binnenkort een onheil voor jou zou worden...

The Dutch is really not a good medium to translate this piece in. There is a French origin, so there was also the idea to translate this in an international language to make it a bit more clear to readers in other countries than the Benelux. You have to take in consideration some words are not t

In [ ]:
import 